In [4]:
import numpy as np
import src.database_generator as dbgr
import time
import h5py
import src.train_surrogate_models as tm
import scipy.interpolate
import warnings
import math
import pickle
warnings.filterwarnings("ignore")

In [ ]:
ind, obj = dg.get_data(['height', 'smear', 'pu_content'], 
                       ['cycle length', 'pu mass', 'reactivity swing', 'burnup'])

In [9]:
with open('../test/sm_lr_2obj.pkl', 'rb') as pickle_file:
    sm = pickle.load(pickle_file)
print('Predicted: {}'.format(sm.predict('lr', [[79.63313,69.95625,0.99652]])))

Predicted: [[242.85502093  39.01472636]]


In [8]:
with open('../test/sm_lr_2var_1const.pkl', 'rb') as pickle_file:
    sm = pickle.load(pickle_file)
print('Predicted: {}'.format(sm.predict('lr', [[79.63313,69.95625,0.99652]])))

Predicted: [[439.55794102  46.45755984   1.25666173]]


In [12]:
with open('../sm_lr_2obj_1const.pkl', 'rb') as pickle_file:
    sm = pickle.load(pickle_file)
print('Predicted: {}'.format(sm.predict('lr', [[79.63313,69.95625,0.99652]])))

Predicted: [[419.57416505  44.26569797]]


In [11]:
#sm = tm.Surrogate_Models()
models = ['lr', 'pr', 'ann', 'gpr', 'rf']
#sm.random = 0

model_dict = {}

for num in range(10):
    model_dict[num] = {}
    for model in models:
        print(model)
        sm = tm.Surrogate_Models()
        sm.update_database(ind, obj)
        sm.optimize_model(model)
        model_dict[num][model] = {'R2': sm.models[model]['score'], 'MSE': sm.models[model]['mse_score']}
        sm.clear_surrogate_model()
    print(model_dict)


lr
pr
ann
gpr
rf
{0: {'lr': {'R2': 0.9247633187623482, 'MSE': 0.05642751092823881}, 'pr': {'R2': 0.9871374977096206, 'MSE': 0.009646876717784612}, 'ann': {'R2': 0.991807967830867, 'MSE': 0.006156345515959851}, 'gpr': {'R2': 0.9694344732833668, 'MSE': 0.022924145037477958}, 'rf': {'R2': 0.9355814569395542, 'MSE': 0.04831390729533437}}}
lr
pr
ann
gpr
rf
{0: {'lr': {'R2': 0.9247633187623482, 'MSE': 0.05642751092823881}, 'pr': {'R2': 0.9871374977096206, 'MSE': 0.009646876717784612}, 'ann': {'R2': 0.991807967830867, 'MSE': 0.006156345515959851}, 'gpr': {'R2': 0.9694344732833668, 'MSE': 0.022924145037477958}, 'rf': {'R2': 0.9355814569395542, 'MSE': 0.04831390729533437}}, 1: {'lr': {'R2': 0.8816213417518889, 'MSE': 0.08878399368608339}, 'pr': {'R2': 0.9840258866269106, 'MSE': 0.011980585029816943}, 'ann': {'R2': 0.9901772034013456, 'MSE': 0.00738624955404747}, 'gpr': {'R2': 0.9857535011103511, 'MSE': 0.01068487416723871}, 'rf': {'R2': 0.9602877760218411, 'MSE': 0.029784167983619138}}}
lr
pr
a

In [24]:
model_dict_2 = {}
for model in ['lr', 'pr', 'ann', 'gpr', 'rf']:
    model_dict_2[model] = []
    s = 0
    for num, models in model_dict.items():
        model_dict_2[model].append(models[model]['MSE'])
    mean = sum(model_dict_2[model])/len(model_dict)
    var  = sum(pow(x-mean,2) for x in model_dict_2[model]) / len(model_dict_2[model])  # variance
    std  = math.sqrt(var)  # standard deviation
    print(model, round(mean,5), round(std,5))

lr 0.06445 0.0103
pr 0.01238 0.00561
ann 0.00979 0.00669
gpr 0.01353 0.00959
rf 0.05861 0.0248


In [ ]:
print('Predicted: {}'.format(sm.predict(model, [[61.37,51.58,0.7340]])))
print('Predicted: {}'.format(sm.predict(model, [[59.72,50.01,0.8694]])))
print('Predicted: {}'.format(sm.predict(model, [[71.06,55.77,0.3536]])))

In [23]:
design_variables = ['height', 'smear', 'pu_content']
objective_functions = ['reactivity swing', 'burnup', 'pu mass', 'cycle length', 
                       'bol keff', 'eol keff', 'void', 'doppler', 'excess reactivity']
model = 'gpr'

df_ind, df_obj = dbgr.get_data(design_variables, objective_functions, 
                               database_name='SFR_DB_3', output='DataFrame', fixed_cycle_length=360, num_cycles=3)
sm = tm.Surrogate_Models()
sm.random = None
sm.update_database(df_ind, df_obj, dataframe=True)
sm.optimize_model(model)
#file_name = open('sm_{}_no_cl.pkl'.format(model), 'wb')
#pickle.dump(sm, file_name)
#file_name.close()
print(sm.models[model]['score'], sm.models[model]['mse_score'])

0.9892731050672201 0.008322373472298849


In [12]:
print('Predicted: {}'.format(sm.predict(model, [[100,80,1.0]])))

Predicted: [[ 3.79756488e+02  7.22961955e+02  2.29390901e+03  2.17679396e+03
   1.00000000e+00 -3.29353581e+01 -3.11951446e-01]]


In [31]:
file_name = open('sm_{}_360days_updated.pkl'.format(model), 'wb')
pickle.dump(sm, file_name)
file_name.close()

In [30]:
print((1.0396-1.01112)/(1.0396*1.01112)*10E5)
print((1.0396-1.01138)/(1.0396*1.01138)*10E5)
print((1.0396-1.01138)/(1.0396*1.01138)*10E5-(1.0396-1.01112)/(1.0396*1.01112)*10E5)

27093.868167508714
26839.62090479232
-254.2472627163952
